In [30]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import os
import random

In [146]:
%%time
# naivebayes 垃圾邮件分类器

# dataset：英文邮件
train_path = "/Users/gunanxi/PycharmProjects/JustForFun/LightGBM/build/LightGBM/python-package/LSTM_learn/MachineLearning/email/train1"
test_path = "/Users/gunanxi/PycharmProjects/JustForFun/LightGBM/build/LightGBM/python-package/LSTM_learn/MachineLearning/email/test1"
     
def load_data(folder_path):
    print("Loading dataset ...")
    datalist = datasets.load_files(folder_path)
    print("summary: {0} documents in {1} categories.".format(len(datalist.data),len(datalist.target_names)))
    return datalist
train = load_data(train_path)
test = load_data(test_path)

def email_num(path,pos):
    list = os.listdir(path + '/' + pos)
    return len(list)
# train集中：正常邮件的数目 和 垃圾邮件的数目
normal = email_num(train_path,"pos")
spam = email_num(train_path,"neg")

Loading dataset ...
summary: 36 documents in 2 categories.
Loading dataset ...
summary: 36 documents in 2 categories.
CPU times: user 4.42 ms, sys: 6.29 ms, total: 10.7 ms
Wall time: 18.5 ms


In [3]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import nltk
import ssl #关闭SSL检查，可以解决无法download nltk.stopwords的问题
 
#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
#    pass
#else:
#    ssl._create_default_https_context = _create_unverified_https_context
 
#nltk.download()
#nltk.download('stopwords')

In [163]:
# nltk 做停用词的小栗子
word_sum = 0
def dropStopwordsAndToken(word_bag,essaydata,word_sum):
    # word_bag:dict
    data = re.sub("[\s+\.\!\/_,$%^*(+\"\'-]+|[+——！，。？、~@#￥%……&*（）<>]+", " ", essaydata)
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(data)

    word_list = [w for w in word_tokens if not w in stop_words]

    for w in word_list: 
        if w not in stop_words:
            w = w.encode("utf-8") #全部转utf-8编码
            w.lower()             #全部转小写
            if w in word_bag:
                word_bag[w] = word_bag[w] + 1
            else:
                word_bag[w] = 1.0
            word_sum = word_sum + word_bag[w]
    return word_bag,word_sum
# testing 
#example_sent = "This is a sample sentence, showing off the stop words filtration."
#word_bag = dropStopwordsAndToken(example_sent)
#print(word_bag) 
word_bag = {}
for s in train.data:
    s = s.decode('utf-8')
    [word_bag,word_sum] = dropStopwordsAndToken(word_bag,s,word_sum)
print("WordNum:",len(word_bag))
print(word_sum)

WordNum: 1888
27537.0


In [265]:
#计算先验概率和条件概率
normal = email_num(train_path,"pos")
spam = email_num(train_path,"neg")

# 正常邮件和垃圾邮件的先验概率
def A_pre_probabilty(data):
    s_pre_pro = []
    #垃圾邮件的先验概率
    P_spam = spam /len(data)
    s_pre_pro.append(P_spam)
    #正常邮件的先验概率
    P_normal = normal/len(data)
    s_pre_pro.append(P_normal)
    
    #返回先验概率的列表
    return s_pre_pro

def Bi_pre_probabilty(word_bag,word_sum):
    for n,w in enumerate(word_bag):
        if( n == 0):
            word_bag.pop(w)
        word_bag[w] = 1.0 * n/word_sum
    return word_bag


#计算每个词在正常邮件垃圾邮件中的数目
def wordNum_email(email_repre,wordDic):
    #用二维向量存储
    num_word = np.zeros((2,len(wordDic)),dtype= np.int)
    for i in range(len(wordDic)):
        #在正常邮件的数目
        for j in range(normal):
            num_word[0][i] += email_repre[j][i]
        #在垃圾邮件中的数目
        for j in range(normal, spam+normal):
            num_word[1][i] += email_repre[j][i]
    print(num_word)
    return num_word

#条件概率
for index in range(len(train.data)):
    s = train.data[index]
    s = s.decode('utf-8')
    if train.target[index] == 0:
        # neg
        [s_word_bag,s_word_sum] = dropStopwordsAndToken(s_word_bag,s,s_word_sum)
    else:
        [n_word_bag,n_word_sum] = dropStopwordsAndToken(n_word_bag,s,n_word_sum)
print(s_word_sum,n_word_sum)

A_prePro = []
if(len(train.data)==0):
    print("No doc!")
else:
    A_prePro = A_pre_probabilty(train.data)
    
if(word_sum==0):
    print("No words!")
else:
    Bi_prePro = Bi_pre_probabilty(word_bag,word_sum)  

n_PABi = Bi_pre_probabilty(n_word_bag,n_word_sum)

s_PABi = Bi_pre_probabilty(s_word_bag,s_word_sum)
print("A_preProbablity:",A_prePro) #0=,1
print("Bi_preProbability:",len(Bi_prePro))
print("n_PABi:",len(n_PABi))
print("s_PABi:",len(s_PABi))


1063151.3150133952 462695.16403066204
A_preProbablity: [0.5, 0.5]
Bi_preProbability: 1889
n_PABi: 1090
s_PABi: 1120


In [276]:
test_sum = len(test.data)
def predict(doc):
    doc = doc.decode('utf-8')
    test_word_bag,test_word_sum = {},0
    [test_word_bag,test_word_sum] = dropStopwordsAndToken(test_word_bag,doc,test_word_sum)
    sps,nps = 0,0
    
    # 是垃圾邮件的概率
    for (n,w) in enumerate(test_word_bag):
        s = 0
        if w in s_PABi and w not in n_PABi:
            s = s + s_PABi[w]*3
        if w in s_PABi and w in n_PABi:
            s = s + (s_PABi[w] - n_PABi[w])*5
        if w not in s_PABi and w in n_PABi:
            s = s - n_PABi[w]*3
        if w in Bi_prePro:
            s = s / Bi_prePro[w]
        sps = sps + s
    # 是正常邮件的概率
    for (n,w) in enumerate(test_word_bag):
        n = 0
        if w in n_PABi and w not in s_PABi:
            n = n + n_PABi[w]*3
        if w in n_PABi and w in s_PABi:
            n = n + (n_PABi[w] - s_PABi[w])*5
        if w in s_PABi and w not in n_PABi:
            n = n - s_PABi[w]*3
        if w in Bi_prePro:
            n = n / Bi_prePro[w]
        n = nps + n
            
    print(sps,nps)
    if sps>nps:
        return 0
    else:
        return 1  # 返回1:n=pos 0:s=neg

acc = 0
for index in range(len(test.data)):
    target = test.target[index]
    pre = predict(test.data[index])
    print(index,target,pre)
    if( target == pre ):
        acc = acc + 1
print(acc,len(test.data))
print("test accuracy:",acc*1.0/len(test.data))


y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
print(accuracy_score(y_true, y_pred))

-176.09352385959556 0
0 1 1
-169.45059427986462 0
1 1 1
17.415065168640556 0
2 0 0
4.6459366469453816 0
3 1 0
5.428462058433862 0
4 1 0
10.695851836897392 0
5 0 0
17.18213082454029 0
6 0 0
3.7899559086914367 0
7 0 0
21.71797081976376 0
8 0 0
26.95882202157036 0
9 1 0
5.965413803004148 0
10 1 0
26.65478769258722 0
11 1 0
9.039752819959864 0
12 1 0
16.599067862070044 0
13 1 0
8.367696570209475 0
14 1 0
3.7080124605251 0
15 0 0
-293.6175622592603 0
16 0 1
7.070053868439077 0
17 0 0
5.108938061784813 0
18 0 0
-6.483248438595487 0
19 0 1
9.990858138423441 0
20 0 0
-0.8439917719417773 0
21 1 1
33.28286486735888 0
22 0 0
13.909564999480562 0
23 0 0
21.955696470066247 0
24 1 0
27.64943722800174 0
25 1 0
22.628587063645636 0
26 0 0
-6.208007825609752 0
27 1 1
6.2432535081744716 0
28 0 0
4.47997954236388 0
29 1 0
15.697731749218356 0
30 1 0
7.399737668697235 0
31 1 0
8.763432751341206 0
32 0 0
5.277443452295082 0
33 1 0
13.100867431176075 0
34 0 0
8.766352787591734 0
35 0 0
20 36
test accuracy: 